In [0]:
# Data lake V2
storage_account_name = "sttpsnown3ebsfdbvgp3o"

# Passando secret scope
secret_scope_name = "kv-scope-tpsnow"

# Criando componentes da autenticação
client_id = dbutils.secrets.get(scope=secret_scope_name, key="databricks-sp-client-id")
tenant_id = dbutils.secrets.get(scope=secret_scope_name, key="databricks-sp-tenant-id")
client_secret = dbutils.secrets.get(scope=secret_scope_name, key="databricks-sp-secret")

# Configurando a sessão do Spark para autenticação
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
# Célula 2 ATUALIZADA - Lendo da Tabela do Unity Catalog 

tabela_bronze_uc = "catalogo_bronze.servicenow.pm_project"

print(f"Lendo dados da tabela do Unity Catalog: {tabela_bronze_uc}")

df_bronze = spark.read.table(tabela_bronze_uc)
df_bronze_amostra = df_bronze.limit(10)

print("Leitura da camada bronze concluída. Exibindo uma amostra de 10 registros:")
display(df_bronze_amostra)

In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date

print("Iniciando transformação dos dados...")

df_silver = df_bronze.select(
    col("number").alias("id_projeto"),
    col("short_description").alias("nome_projeto"),
    col("project_manager").alias("gerente_projeto"),
    col("state").alias("estado_projeto"),
    to_timestamp(col("sys_created_on")).alias("data_criacao_utc"),
    to_date(col("start_date")).alias("data_inicio"),
    to_date(col("end_date")).alias("data_fim"),
    col("work_cost").cast("decimal(18, 2)").alias("custo_trabalho"),
)

print("Transformação concluída:")
display(df_silver)

In [0]:
# salva na camada silver

silver_path = f"abfss://silver@{storage_account_name}.dfs.core.windows.net/servicenow/projetos"

print(f"Salvando dados transformados na camada silver em: {silver_path}")

df_silver.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(silver_path)

spark.sql(f"CREATE TABLE IF NOT EXISTS catalogo_silver.servicenow.projetos LOCATION '{silver_path}'")

print("Dados salvos com sucesso na camada silver e tabela registrada no catálogo!")
print("Verificando os dados lendo da nova tabela silver...")
display(spark.read.table("catalogo_silver.servicenow.projetos"))